In [1]:
import permutationTest as pt

In [2]:
#from utils2 import getSynteticData, SNSMultipleboxPlot, ttests, MWU, exactTest, getErrors
import numpy as np
from scipy.stats import mannwhitneyu, ttest_ind
import permutationTest as pt
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
%load_ext cython
%load_ext rpy2.ipython


In [ ]:
%%R

install.packages("coin")

install.packages("ggplot2")

install.packages("perm")

install.packages("exactRankTests")


library(ggplot2)
library(coin)

In [ ]:
%%R
    runMcCoin <- function(x,y,batches,output,n_samples) {
       e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
            
            DV <- c(x[[i]], y[[i]])
            IV <- factor(rep(c("A", "B"), c(length(x[[i]]), length(y[[i]]))))
                    # for oneway_test(), pvalue()
            output[[i]] <- pvalue(oneway_test(DV ~ IV, distribution="approximate"))
            #output[[i]] <- pvalue(oneway_test(DV ~ IV, distribution="split-up""))
      
            
      
            
            }
    

    return(output)
        
    }

In [ ]:
def McCoin(A,B, batches):
    x = pd.DataFrame(A, columns=list(range(A.shape[1]))).T
    y = pd.DataFrame(B, columns=list(range(B.shape[1]))).T
    n_samples = x.shape[1]
    outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T
    
    %R -i x
    %R -i y
    %R -i batches
    %R -i outputDf
    %R -i n_samples
    
    
    %R out <- runMcCoin(x,y, batches, outputDf, n_samples)
    
    
    %R -o out
    return list(out.T["output"].values)



In [ ]:
def getSynteticData(func, setN=10, sampleN=2_000, mean=0, std=1,seed=1):
    """Generate synthetic data"""
    np.random.seed(seed)
    AN, BN = [func(mean,std,setN) for i in range(sampleN)], [func(0,std,setN) for i in range(sampleN)]
    return AN, BN

def getErrors(pValList, pTtestList, rel=True):
    relatvieErrorList = list()
    for pv, pt in zip(pValList, pTtestList):
        if rel:
            relatvieErrorList.append(relError(pv, pt))
        else:
            relatvieErrorList.append(frac(pv, pt))
            
    return relatvieErrorList

def relError(x,y):
    return frac((x - y), y) 
def frac(x,y):
    return x / y

In [ ]:
def exactTest(A,B, bins=10, one_side=False):
    return pt.GreenFloatCuda(A,B,bins)
    
    
def MWU(A, B, one_side=False):
    p_mw = list()
    for a,b in zip(A, B):
        if one_side:
            p_mw.append(mannwhitneyu(a,b, alternative="less")[1])
        else:
            p_mw.append(mannwhitneyu(a,b, alternative="two-sided")[1])
    return p_mw

def ttests(A,B, one_side=False):
    p_t = list()
    for x, y in zip(A, B):
        t, p = ttest_ind(y, x)
        if one_side:
            p = p/2
            if t<0:
                p = 1-p
        p_t.append(p)
    return p_t

In [ ]:
999999+1

In [ ]:
100000

In [ ]:
10**-6

In [ ]:
def getPvalues(mean=0.2):
    allFracList1 = list()
    allFracMWUList1 = list()
    allFracFPList1 = list()
    for setS in setSize:
        Anorm0, Bnorm0 = getSynteticData(np.random.normal, mean=mean, setN=setS,sampleN=50)
        
        ptTest  = ttests(Anorm0, Bnorm0)
        
        peList = exactTest(np.asarray(Anorm0), np.asarray(Bnorm0), bins=30)

        ExactFracList = getErrors(peList, ptTest, False) 
        allFracList1.append(np.log10(ExactFracList))
    
        pmwList = MWU(Anorm0, Bnorm0)
        MWUFracList = getErrors(pmwList, ptTest, False) 
        allFracMWUList1.append(np.log10(np.abs(MWUFracList)))
    

        ptfp = McCoin(np.array(Anorm0), np.array(Bnorm0),999999)
        FPFracList = getErrors(ptfp, ptTest, False)
        allFracFPList1.append(np.log10(FPFracList))
    
    #return allFracList1, allFracMWUList1, allFracFPList1
    return ptTest, peList, pmwList, ptfp

def getNwPlot(setSize, mean=0.1):
    N_w = list(range(5, 105,5))
    allEerrorNw, allEerrorNwFrac = list(), list()
    for n_w in N_w:
        Anorm0, Bnorm0 = getSynteticData(np.random.normal, mean=mean, setN=setSize,sampleN=50)
        ptTest  = ttests(Anorm0, Bnorm0)
    
        peList = exactTest(Anorm0, Bnorm0, bins=n_w)
        ExactErrorList = getErrors(peList, ptTest)
        ExactFracList = getErrors(peList, ptTest, False) 
        allEerrorNw.append(np.log10(np.abs(ExactErrorList)))
        allEerrorNwFrac.append(np.log10(ExactFracList))
    return allEerrorNwFrac, N_w
    
    

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def prepareBoxPandas(timeData, sampleSizes,label=None):
    preparePd = list()
    for time, sample in zip(timeData, sampleSizes):
        for t in time:
            if label:
                preparePd.append([t, sample, label])
            else:
                preparePd.append([t, sample])
    return preparePd

def SNSMultipleboxPlot(allEerrorList, Bin, allMWUList=None, allFPList=None, log=True, 
                       path=None, test_type="setSize", relError=True, dashed=False,
                       ylim=False):
    
    plt.figure(figsize=(16, 10))
    
    dataParallel = prepareBoxPandas(allEerrorList, Bin, "Parallel Green")
    
    data= dataParallel
    
    if allFPList:
        dataFastApprox = prepareBoxPandas(allFPList, Bin, "FastPerm")
        data += dataFastApprox
    if allMWUList:
        MannWhitneyApprox = prepareBoxPandas(allMWUList, Bin, "Mann–Whitney $\it{U}$ test")
        data += MannWhitneyApprox
    
    sns.set(style="white")
    sns.set_context("talk")
    
    pdData = pd.DataFrame(data, columns=['error','bins', 'Method'])
    
    if test_type=="setSize":
        snsPlot = sns.boxplot(x="bins", y="error", data=pdData, hue="Method")
    else:
        snsPlot = sns.boxplot(x="bins", y="error", data=pdData, color="skyblue")
        
    
    if relError:
        plt.ylabel(r"Relative error $\big(|\frac{p_{*}-p_{t}}{p_{t}}|)$",fontsize=25)
    else:
        plt.ylabel(r"$\frac{p_{*}}{p_{t}}$",fontsize=45)
        
    if test_type=="windowSize":
        plt.xlabel(r"$n_{w}$",fontsize=45)
    else:
        plt.xlabel(r"$n$",fontsize=45)
    
    sns.set_style("ticks")
    sns.despine()
    plt.tight_layout()
    plt.gcf().subplots_adjust(left=0.30)
    
     
    
    if log:
        if allFPList and allMWUList:
            MAX = max(np.max(allEerrorList), np.max(allFPList), np.max(allMWUList))
            MIN = min(np.min(allEerrorList), np.min(allFPList), np.min(allMWUList))
        elif allFPList:
            MAX = max(np.max(allEerrorList), np.max(allFPList))
            MIN = min(np.min(allEerrorList), np.min(allFPList))
        elif allMWUList:
            MAX = max(np.max(allEerrorList), np.max(allMWUList))
            MIN = min(np.min(allEerrorList), np.min(allMWUList))
        else:
            MAX = np.max(allEerrorList)
            MIN = np.min(allEerrorList)
         

        if dashed:
            plt.axhline(0, ls=':', color="black", linewidth=3)
        
            
        #RANGE = np.arange(np.floor(MIN), np.ceil(MAX), 100)
        RANGE = np.arange(np.floor(MIN), np.ceil(MAX))
        
        if not relError:
            if ylim and ylim[1]<10:
                RANGE = np.array(sorted(list(RANGE) + [np.log10(0.5), np.log10(2)]))
            else:
                RANGE = np.array(sorted(list(RANGE)))

        plt.yticks(RANGE, 10.0**RANGE)

        
        
    if test_type=="setSize":
        plt.legend().set_title('')
    
    #€plt.yticks([0.5, 1, 2])
    
    
    if ylim:
        plt.ylim(np.log10(ylim[0]) ,np.log10(ylim[1]))
        path += "_zoom"
        
        plt.xticks(size = 32)
        plt.yticks(size = 32)
    else:
        plt.xticks(size = 32)
        plt.yticks(size = 25)
        
    

    
    if path:
        fig = snsPlot.get_figure()
        fig.savefig(path)

In [ ]:
setSize = [10,50,100,150,200,250,300]

In [ ]:
ptTest, peList, pmwList, ptfp = getPvalues()

In [ ]:
ptTest

In [ ]:
ptfp

In [ ]:
ptfp

In [ ]:
for pv, pt in zip(ptTest, peList):
    print(np.log10(pv / pt))

In [ ]:
ptTest

In [ ]:
!ls "./figures/accuracy/accuracy/"

In [ ]:
SNSMultipleboxPlot(allFracList1, setSize, allFracMWUList1, 
                   allFracFPList1, 
                   relError=False,
                   path="./figures/accuracy/02",
                  dashed=True,ylim=[0.4, 2.1])

In [ ]:
allFracList2, allFracMWUList2, allFracFPList2 = getPvalues(1)

In [ ]:
SNSMultipleboxPlot(allFracList2, setSize, allFracMWUList2, 
                   allFracFPList2, relError=False,path="./figures/accuracy/comparison/1", 
                   dashed=True, ylim=[10**-3, 10**3])



# |A|=|B|=150 $\mu = 1$

In [ ]:
n3 = setSize[3]; n3

In [ ]:
allEerrorNwFrac3_m_1, N_w = getNwPlot(n3, 1)

SNSMultipleboxPlot(allEerrorNwFrac3_m_1[1:], N_w[1:], test_type="windowSize", 
                   relError=False,path="./figures/accuracy/windowSize/1_" + str(n3),
                  dashed=True)

In [ ]:
SNSMultipleboxPlot(allEerrorNwFrac3_m_1[1:], N_w[1:], test_type="windowSize", 
                   relError=False,path="./figures/accuracy/windowSize/1_" + str(n3),
                  dashed=True, ylim=[0.4, 2.1])

# |A|=|B|=150 $\mu = 0.2$

In [ ]:
allEerrorNwFrac3_m_02, N_w = getNwPlot(n3, 0.2)

SNSMultipleboxPlot(allEerrorNwFrac3_m_02[1:], N_w[1:], test_type="windowSize", 
                   relError=False,path="./figures/accuracy/windowSize/02_" + str(n3),
                  dashed=True)

In [ ]:
SNSMultipleboxPlot(allEerrorNwFrac3_m_02[1:], N_w[1:], test_type="windowSize", 
                   relError=False,path="./figures/accuracy/windowSize/02_" + str(n3),
                  dashed=True, ylim=[0.4, 2.1])